## MPI4Py example

In this example we use [IPCMagic](https://github.com/eth-cscs/ipcluster_magic/tree/master) to run an example from [MPI4Py's homepage](https://mpi4py.readthedocs.io/en/stable/tutorial.html#point-to-point-communication) within a Jupyter notebook.

* From a shell in Piz Daint this can be run using this Slurm job script:
 
```
#!/bin/bash -l

#SBATCH --job-name=osubw
#SBATCH --time=00:05:00
#SBATCH --nodes=2
#SBATCH --ntasks-per-core=1
#SBATCH --ntasks-per-node=1
#SBATCH --cpus-per-task=12
#SBATCH --partition=normal
#SBATCH --constraint=gpu
#SBATCH --account=<project>

# source python environment with cupy and mpi4py

srun python example.py
```

In [ ]:
import os
import ipcmagic

In [ ]:
%ipcluster start -n 2

In [ ]:
%%px
import socket

socket.gethostname()

In [ ]:
# Disable IPyParallel's progress bar
%pxconfig --progress-after -1

In [ ]:
%%px
from mpi4py import MPI
import numpy

In [ ]:
%%px
comm = MPI.COMM_WORLD
rank = comm.Get_rank()

In [ ]:
%%px

# passing MPI datatypes explicitly
if rank == 0:
    data = numpy.arange(10, dtype='i')
    comm.Send([data, MPI.INT], dest=1)
elif rank == 1:
    data = numpy.empty(10, dtype='i')
    comm.Recv([data, MPI.INT], source=0)
    print(f' * array from rank {rank}: {data}')

In [ ]:
%%px

# automatic MPI datatype discovery
if rank == 0:
    data = numpy.arange(10, dtype=numpy.float64)
    comm.Send(data, dest=1)
elif rank == 1:
    data = numpy.empty(10, dtype=numpy.float64)
    comm.Recv(data, source=0)
    print(f' * array from rank {rank}: {data}')

In [ ]:
%ipcluster stop